In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

***Question 1***

La charge de la batterie est très importante afin de connaître l'autonomie du système. On a 
$$
I = \dfrac{\partial Q}{\partial t}
$$
En discrétisé on a, à l'instant i 
$$
I_i = \frac{Q_i}{\Delta t}
$$
Ainsi on trouve bien que pour tous les instants entérieurs, on doit obtenir que 
$$
\sum_{i=0}^{N}\Delta t I_i \leq Q_bat
$$

***Question 2***

$a_1$ : paramétre de frottement statique. Ce paramètre varie en fonction du poids du cycliste

$a_2$ : terme de frottement de l'air ( hypothèse d'écoulement laminaire). Ce paramètre varie en fonction de la surface du cycliste

$a_3$ : terme de couple moteur. Ne varie pas en fonction du cycliste

$a_4$ : terme de couple humain ( couple du cycliste sur les pédales ). Varie en fonction de la force musculaire du cycliste

$-g\gamma (x_i)$ : terme de résistance de la pesenteur au mouvement. Ne dépend pas du cycliste car la masse a été éliminé pour obtenir ces équations

***Question 3***

On veut une assistance optimale, c'est à dire qui permettent d'aller le plus loin possible sans avoir à s'épuiser. Il faut donc chercher à minimiser $\sum_{i=0}^{N}T_i ^2$ ( le couple musculaire ), en farantissant la condition de batterie $\sum_{i=0}^{N}\Delta t I_i \leq Q_{bat}$.

Le problème d'optimisation se réécrit donc :
$$
\underset{T_i,I_i,x_i,v_i,t_i}{min} \sum_{i=0}^{N}T_i ^2  \
$$
Tel que 
$$
v_{i+1} - v_i =  \Delta t(a_1 - a_2vi + a_3Ii + a_4Ti + g\gamma (xi))\\
\sum_{i=0}^{N}\Delta t I_i - Q_{bat} = 0 \\
I_i - I_M \leq 0
$$

***Question 4***

Etude de la convexité du problème : \
On pose
$$
f : (T_i,I_i,x_i,v_i,t_i) \rightarrow \underset{T_i,I_i,x_i,v_i,t_i}{min} \sum_{i=0}^{N}T_i ^2
$$
Si l'on considère le couple de pédalage constant, on a :
$$
\nabla ^2 f(T_i,I_i,x_i,v_i,t_i) = 0 
$$
Donc 
$$
\nabla ^2 f(T_i,I_i,x_i,v_i,t_i) \ge 0 
$$
Ainsi f est bien convexe.

Bon c'est clairement faux mais je ne savais pas quoi mettre, sinon je dis juste que ça depend de la tête de Ti

***Identification du modèle dynamique***

***Question 5***

Formulation du problème des moindres carrés: \
On pose,

$$
A = 
\begin{pmatrix}
1 & -v_0 & I_0 & T_0 \\
1 & -v_1 & I_1 & T_1 \\
... & ... & ... & ... \\
1 & -v_n & I_n & T_n \\
\end{pmatrix} 
$$

$$
B = 
\begin{pmatrix}
v_1 - v_0 \\ v_2 - v_1 \\ ... \\ v_{n+1} - v{n}
\end{pmatrix}
$$

$$
X = \begin{pmatrix}
a_1 \\ a_2 \\ a_3 \\ a_4
\end{pmatrix}
$$

Le problème des moindres carrés s'écrit alors :

$$
AX = B
$$

***Question 6***

In [2]:
with open('data_velo.csv', encoding = "utf8") as f :
    df = pd.read_csv(f)
    #print(df.head())
    p = len(df.columns) - 1
    n = len(df.axes[0])
    A = np.zeros((n,p))
    B = np.zeros((n,1))
    V = df['Vitesse [m/s]']
    I = df['I [A]']
    T = df['Couple pédale [Nm]']
    dt = 0.01
    #print(V)
    for i in range(len(V) - 1):
        B[i] = V[i+1] - V[i]
    for j in range(n):
        A[j][0] = 1
        A[j][1] = -V[j]
        A[j][2] = I[j]
        A[j][3] = T[j]
    A = A * dt

    X = np.linalg.lstsq(A, B, rcond = -1)[0]
    print(f"Les solutions obtenues par la méthodes des moindres carrés sont a1 = {X[0]}, a2 = {X[1]}, a3 = {X[2]}, a4 = {X[3]}")

Les solutions obtenues par la méthodes des moindres carrés sont a1 = [-0.29498251], a2 = [0.021807], a3 = [0.02869084], a4 = [0.0143385]


***Question 7***

***a)***

On pose :
$$
C = A_k^TA_k \\
U = a_{k+1} \\
V = a_{k+1}^T \\
\bar{C} = C + UV^T
$$
On suppose $\bar{C}$ inversible, on peut donc appliquer la formule de Sherman-Morrison-Woodbury :
$$
\bar{C}^{-1} = C^{-1} - C^{-1}U(I + V^TA^{-1}U)^{-1}V^TA^{-1}
$$
on trouve finalement bien que :
$$
    (A_k^T A_k + a_{k+1}a_{k+1}^T)^{−1} = (A_k^T A_k)^{−1} − K_k a_{k+1}^T(A_k^T A_k)^{−1}
$$
Avec :
$
Kk = γk(A_k^T A_k)^{−1}a_{k+1} 
$ \
où 
$
γk = 1/(1 + a_{k+1}^T(A_k^T A_k)^{−1}a_{k+1}) 
$
\
Or par multiplication de matrice :
$$
A_{k+1}A_{k+1}^T = (A_k^T, ak+1)^T(A_kT, ak+1) = A_kA_k^T + a_{k+1}^Ta_{k+1}
$$
On a finalement :
$$
(A_{k+1}^TA_{k+1})^{−1} = (A_k^T A_k + a_{k+1}a_{k+1}^T)^{−1}
$$
D'où :
$$
\boxed{
(A_{k+1}^TA_{k+1})^{−1} = (A_k^T A_k + a_{k+1}a_{k+1}^T)^{−1} = (A_k^T A_k)^{−1} − K_k a_{k+1}^T(A_k^T A_k)^{−1}}
$$


***b)***

On sait que : 
$$
x_k = (A_{k}^TA_{k})^{−1}A_{k}^{T}b_{k}
$$
Alors on déduit:
$$
\begin{align*}
x_{k+1} &= (A_{k+1}^TA_{k+1})^{−1}A_{k+1}^{T}b_{k+1} \\
&= [(A_{k}^TA_{k})^{−1} − K_k a_{k+1}^T(A_k^T A_k)^{−1}][A_{k}^{T}b_{k} + a_{k+1} \tilde{b}] \\ 
&= x_k + K_k \tilde{b} (1 + a_{k+1}^T(A_k^T A_k)^{−1}a_{k+1}) - K_k a_{k+1}^{T} [(A_k^{T}A_k)^{-1}A_k^T b_k]-  K_k \tilde{b} a_{k+1}^{T}(A_k^T A_k)^{-1} a_{k+1} \\ 
&= x_k + K_k (\tilde{b} - a_{K+1}^{T} x_k)
\end{align*}
$$

On a alors identifié $x_k = (A_k^{T}A_k)^{-1}A_k^T b_k$ dans l'avant-dernier terme, le dernier se simplifiant avec un des deux premiers.

D'où notre résultat:
$$
\boxed{x_{k+1} = x_k + K_k (\tilde{b} - a_{K+1}^{T} x_k)}
$$

***c)***

On peut remarquer qu'en posant $U_{k} = (A_{k}^{T} A_k)^{-1}$, alors on obtient selon les formules précédentes $U_{k+1}$ sans avoir besoin d'inverser de matrice. 

Ainsi, nous allons baser notre nouvel algorithme là-dessus, en prenant un $x_0$ proche de la solution des moindres carrés :

In [3]:
#Paramétrage initial
A0 = A[:4, :5]
Uk = inv(np.dot(A0, np.transpose(A0)))
xk = np.array([-0.1, 0.01, 0.01, 0.01])
N = 3251 #Nombre de données

***Question 8***

In [4]:
#On itère N fois
for k in range(N):
    ak = A[k, :]
    bk = B[k]
    Kk = np.dot(Uk, ak) / (1 + np.dot(np.dot(np.transpose(ak),Uk),ak))
    Uk = Uk - np.dot(np.dot(Kk,np.transpose(ak)),Uk)
    xk = xk + Kk*(bk - np.dot(np.transpose(ak),xk))

#Affichage du résultat final:
print(xk)

[-0.09632364  0.00216428  0.00917734  0.01499101]


On trouve au final des résultats différents avec la question 6, mais on peut déjà avoir "en direct" des résultats plutôt fiables.
En effet, si on prend le résultat à diverses périodes alors que nous n'avons pas pris en compte toutes les valeurs, on obtient :

In [5]:
for k in range(N):
    ak = A[k, :]
    bk = B[k]
    Kk = np.dot(Uk, ak) / (1 + np.dot(np.dot(np.transpose(ak),Uk),ak))
    Uk = Uk - np.dot(np.dot(Kk,np.transpose(ak)),Uk)
    xk = xk + Kk*(bk - np.dot(np.transpose(ak),xk))
    if k in [500, 1000, 1500, 2000, 2500, 3000]:
        print(xk)

[-0.09664489  0.00284884  0.00924915  0.0145551 ]
[-0.09688067  0.00335129  0.00930186  0.01423515]
[-0.09659982  0.00275281  0.00923908  0.01461625]
[-0.09639987  0.00232672  0.00919438  0.01488758]
[-0.09631248  0.0021405   0.00917485  0.01500615]
[-0.09618496  0.00186877  0.00914635  0.01517918]


Ainsi, on obtient de bons résultats en direct, c'est une progression !

***Question 9***

En supposant la pente nulle, on annule l'effet de la force de pesanteur car la réaction du support compense parfaitement celle-ci.

In [6]:
#Paramétrage:
a1 = -0.2949
a2 = 0.0218
a3 = 0.02869
a4 = 0.01433
T = 500
dt = 2.5
x0 = 0
xf = 500
Qbat = 500
IM = 30
g = 9.81
N = int(T / dt)

In [8]:
from casadi import *
import time

ModuleNotFoundError: No module named 'casadi'

In [9]:
#Optimisation du premier cas: Pente nulle
opti = casadi.Opti();

v = opti.variable(N)
Ii = opti.variable(N)
T = 0

for i in range(N - 1):
    T += (((v[i + 1] - v[i]) / (dt) - a1 + a2 * v[i] - a3 * Ii[i]) / a4)**2
    
opti.minimize(T)

Q = 0

for i in range(N - 1):
    a = Ii[i]
    opti.subject_to(Ii[i] <= IM)
    opti.subject_to(Ii[i] >= 0)
    Q += dt * Ii[i]
     
opti.subject_to(Q <= Qbat)

v0 = np.zeros(N)
I0 = 5.74 * np.ones(N)
opti.set_initial(v, v0) #On part à une vitesse nulle
opti.set_initial(Ii, I0)

opti.solver('ipopt');
sol = opti.solve();

NameError: name 'casadi' is not defined

In [ ]:
print(sol.value(v), sol.value(Ii))

[ 11.54908584  10.25373857   9.02900988   7.87104983   6.7762184
   5.74107399   4.76236265   3.8370078    2.96210061   2.13489081
   1.35277809   0.61330388  -0.08585635  -0.7469004   -1.37190624
  -1.96283859  -2.52155503  -3.04981187  -3.54926968  -4.02149851
  -4.46798279  -4.89012606  -5.2892553   -5.66662517  -6.02342194
  -6.36076719  -6.67972135  -6.98128707  -7.2664123   -7.53599333
  -7.79087759  -8.0318663   -8.25971701  -8.47514597  -8.67883037
  -8.87141048  -9.0534917   -9.22564637  -9.38841568  -9.54231128
  -9.68781694  -9.82539006  -9.9554631  -10.07844493 -10.19472216
 -10.30466029 -10.40860492 -10.5068828  -10.59980286 -10.68765719
 -10.77072196 -10.84925829 -10.92351305 -10.99371967 -11.06009883
 -11.1228592  -11.18219806 -11.23830195 -11.29134722 -11.34150063
 -11.38891983 -11.43375388 -11.47614373 -11.51622261 -11.55411652
 -11.58994457 -11.6238194  -11.65584748 -11.6861295  -11.71476064
 -11.74183092 -11.76742542 -11.7916246  -11.81450453 -11.83613713
 -11.856590

***Question 10***

In [ ]:
#Nouveaux paramètres
g1 = 0.02
g2 = 0.01
s1 = 100
s2 = 31.5
m1 = 187.5
m2 = 375

In [ ]:
def gamma(x):
    return g1 * np.exp(-((x - m1)**2)/ (s1)**2) + g2 * np.exp(-((x - m2)**2)/ (s2)**2)

In [ ]:
def gammad(x):
    return g1 * (-2*(x - m1)/((s1)**2)) * np.exp(-((x - m1)**2)/ (s1)**2) + g2 * (-2*(x - m2)/((s2)**2)) * np.exp(-((x - m2)**2)/ (s2)**2)

In [ ]:
#Optimisation du second cas: Pente non nulle
opti = casadi.Opti();

x = opti.variable(N, 2)
v = opti.variable(N)
Ii = opti.variable(N)
T = 0

for i in range(N - 1):
    T += (((v[i + 1] - v[i]) / (dt) - a1 + a2 * v[i] - a3 * Ii[i] + g * (gamma(x[i][0]) + gammad(x[i][0])* (x[i][1] - x[i][0]))) / a4)**2
    x[i + 1][0] = x[i][1]
    
opti.minimize(T)

Q = 0

for i in range(N - 1):
    a = Ii[i]
    opti.subject_to(Ii[i] <= IM)
    opti.subject_to(Ii[i] >= 0)
    Q += dt * Ii[i]
     
opti.subject_to(Q <= Qbat)
opti.subject_to(x[N][1] == xf)

v0 = np.zeros(N)
I0 = 5.74 * np.ones(N)
x0_0 = np.array([x0, 0.])

opti.set_initial(x[0], x0_0)
opti.set_initial(v, v0) #On part à une vitesse nulle
opti.set_initial(Ii, I0)

opti.solver('ipopt');
sol = opti.solve();

RuntimeError: .../casadi/core/mx.cpp:402: Assertion "in_range(kk.nonzeros(), -sz+ind1, sz+ind1)" failed:
Out of bounds error. Got elements in range [1,1], which is outside the range [-1,1).